# Question
The following figures show the Di Wang Tower in Shenzhen. The structure of Di Wang Tower is made up of steel and reinforced concrete (calculated as reinforced concrete), C terrain type, the design wind pressure $w_0 = 0.75 \text{ kN/m}^2$, period $T_1=6.38\text{ s}$, The first mode shape is listed in following table. To determine shape factor, the cross section could be considered as rectangle.

<center>
    <img src="./images/Diwang01.png" alt="The Elvation of the DiWang Tower" width="400"/>
    <img src="./images/Diwang02.png" alt="The Plan View of the DiWang Tower" width="400"/>
</center>

<br>

| $\dfrac{h}{H}$ | 0     | 0.1   | 0.2   | 0.3   | 0.4    | 0.5   | 0.6   | 0.7    | 0.8   | 0.9   | 1.0   |
| -------------- | ----- | ----- | ----- | ----- | ------ | ----- | ----- | ------ | ----- | ----- | ----- |
| Mode shape     | 0.0000| 0.0137| 0.0558| 0.1277| 0.2424| 0.3503| 0.4629| 0.5903| 0.7309| 0.8700| 1.0000| 
| Mass / ton     | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 2500  | 1500  | 

Only consider the first mode, calculate **the extreme of top displacements** induced by the along-wind static and dynamic wind load in the given wind direction using the following methods:
1. Method recommended by the Chinese code for wind-resistant design of buildings (GB 50009-2012).
2. According to the quasi-steady assumption, convert the fluctuating wind velocity time histories in `windData` (**with a duration of 10 minutes and a sampling frequency of 10 Hz**) to wind load time histories $F_i(t)=[1/2\rho \bar{u_i}^2+\rho \bar{u_i} u'_i(t)] \cdot \mu_{s,i} \cdot A_i$, and then using stocastic vibration methods in time domain (such as the newmark-beta algorithm) and in frequency domain (such as the response spectrum analysis). Peak factor $g = 2.5$ should be used for the extreme value calculation, and the programming using frequency domain method is optional.

# Answer1 (GB 50009-2012)


In [ ]:
import math
import numpy as np
H = 325 #帝王大厦总高度
z = [32.5,65,97.5,130,162.5,195,227.5,260,292.5,325] # 集中质量分布高度，帝王大厦总高约325m
w0 = 0.75 # 基本风压
B = 65.88 # 迎风面宽度
D = 35.50 # 迎风厚度
T1 = 6.38 # 自振周期
f1 = 1 / T1 # 自振频率
phi1_z = np.array([0.0137,0.0558,0.1277,0.2424,0.3503,0.4629,0.5903,0.7309,0.8700,1.0000]) # 第一振型振型系数

# 计算每个集中质量处迎风面积
s1 = B * 32.5 * 1.5
s2 = B * 32.5 * 1
s3 = B * 32.5 * 1
s4 = B * 32.5 * 1
s5 = B * 32.5 * 1
s6 = B * 32.5 * 1
s7 = B * 32.5 * 1
s8 = B * 32.5 * 1
s9 = B * 32.5 * 0.5 + ((32.5-21.49)* B + 23.65 * 6.49 * 2 + 12* 15 *2) / 2
s10 = ((32.5-21.49)* B + 23.65 * 6.49 * 2 + 12* 15 *2) / 2
s = [s1,s2,s3,s4,s5,s6,s7,s8,s9,s10]

# 计算每个集中质量处风压高度变换系数
μz = []
for i in range(0,10):
    a = 0.544 * (z[i] / 10) ** 0.44
    μz.append(a)

# 每个集中质量处风荷载体型系数
μs = [1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4] #近似为矩形截面，体形系数迎风面0.8，背风面-0.6

# 计算每个集中质量处阵风系数βz 
kw = 0.54 # C类场地地面粗糙度
ξ1 = 0.05 # 钢筋混凝土结构阻尼比
x1 = 30 * f1 / ((kw * w0)**0.5)
R = math.sqrt((math.pi / (6 * ξ1)) * (x1**2 / (1 + x1**2)**(4/3))) #脉动风荷载的共振分量因子
k = 0.295 #系数
a1 = 0.261 #系数
rho_z = 10 * math.sqrt(H + 60 * math.exp(-H / 60) - 60) / H #脉动风荷载竖直方向的相关系数
rho_x = 10 * math.sqrt(B + 50 * math.exp(-B / 50) - 50) / B #脉动风荷载水平方向的相关系数
Bz = [] #脉动风荷载的背景分量因子
for i in range(0,10):
    a = k * (H ** a1) * rho_x * rho_z * phi1_z[i] / μz[i]
    Bz.append(a)

g = 2.5 # 峰值因子
I10 = 0.23 # 10m高度名义湍流强度，C类场地取0.23
βz = [] # 阵风系数
for i in range(0,10):
    a = 1 + 2 * g * I10 * Bz[i] * (1+R**2)**0.5
    βz.append(a)
# 计算每个集中质量处的风荷载标准值
wk = []
for i in range(0,10):
    a = βz[i] * μs[i] * μz[i] * w0
    wk.append(a)

# 计算每个集中质量处的集中力
F = np.array(wk) * np.array(s)

M_matrix = np.array([[2500,0,0,0,0,0,0,0,0,0],
                    [0,2500,0,0,0,0,0,0,0,0],
                    [0,0,2500,0,0,0,0,0,0,0],
                    [0,0,0,2500,0,0,0,0,0,0],
                    [0,0,0,0,2500,0,0,0,0,0],
                    [0,0,0,0,0,2500,0,0,0,0],
                    [0,0,0,0,0,0,2500,0,0,0],
                    [0,0,0,0,0,0,0,2500,0,0],
                    [0,0,0,0,0,0,0,0,2500,0],
                    [0,0,0,0,0,0,0,0,0,1500]]) # 集中质量阵
phi1_z_matrix = phi1_z.reshape(-1, 1)
M1 = phi1_z_matrix.T @ M_matrix @ phi1_z_matrix # 一阶模态质量
F_matrix = F.reshape(-1,1) #集中力矩阵
F1 = phi1_z_matrix.T @ F_matrix #一阶模态集中力
n1 = F1[0,0] / M1[0,0] / (2 * math.pi * f1)**2 #一阶模态坐标
x_all = n1 * phi1_z #各集中质量处的位移
x_max = x_all[-1]
print(f"使用GB50009-2012中的方法计算得出地王大厦顶部位移极值为: {x_max} m")

# Answer2 (时域方法)

In [ ]:
import numpy as np
import scipy.io as sio
import math
import matplotlib.pyplot as plt
simDataPath = 'D:\下载\windData.mat'
simData = sio.loadmat(simDataPath, squeeze_me=True, struct_as_record=False)
U = simData['U']
Z = simData['Z']
dt = simData['dt']
t = simData['t']
del simData
U = U[1:,:]

B = 65.88 # 迎风面宽度
D = 35.50 # 迎风厚度
T1 = 6.38 # 自振周期
f1 = 1 / T1 # 自振频率
w1 = 2 * math.pi * f1 #自振圆频率
ξ1 = 0.05 # 钢筋混凝土结构阻尼比

phi1_z = np.array([0.0137,0.0558,0.1277,0.2424,0.3503,0.4629,0.5903,0.7309,0.8700,1.0000]) # 第一振型振型系数
M_matrix = np.array([[2500000,0,0,0,0,0,0,0,0,0],
                    [0,2500000,0,0,0,0,0,0,0,0],
                    [0,0,2500000,0,0,0,0,0,0,0],
                    [0,0,0,2500000,0,0,0,0,0,0],
                    [0,0,0,0,2500000,0,0,0,0,0],
                    [0,0,0,0,0,2500000,0,0,0,0],
                    [0,0,0,0,0,0,2500000,0,0,0],
                    [0,0,0,0,0,0,0,2500000,0,0],
                    [0,0,0,0,0,0,0,0,2500000,0],
                    [0,0,0,0,0,0,0,0,0,1500000]]) # 集中质量阵

# 计算每个集中质量处迎风面积
s1 = B * 32.5 * 1.5
s2 = B * 32.5 * 1
s3 = B * 32.5 * 1
s4 = B * 32.5 * 1
s5 = B * 32.5 * 1
s6 = B * 32.5 * 1
s7 = B * 32.5 * 1
s8 = B * 32.5 * 1
s9 = B * 32.5 * 0.5 + ((32.5-21.49)* B + 23.65 * 6.49 * 2 + 12* 15 *2) / 2
s10 = ((32.5-21.49)* B + 23.65 * 6.49 * 2 + 12* 15 *2) / 2
s = [s1,s2,s3,s4,s5,s6,s7,s8,s9,s10]

# 每个集中质量处风荷载体型系数
μs = [1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4,1.4] # 近似为矩形截面，体形系数迎风面0.8，背风面-0.6

# 将风速时程转化为力时程
F = []
for i in range(0,10):
    u_average = np.mean(U[i,:])
    u_fluctuate = U[i,:] - u_average
    wk = np.array(u_fluctuate) * u_average * 1.25 + 0.5 * 1.25 * (u_average ** 2)
    a = wk * μs[i] * s[i]
    F.append(a)
F = np.array(F)

F1 = phi1_z.T @ F
M1 = phi1_z.T @ M_matrix @ phi1_z

#求解动力学方程（使用线加速度法）
m = 1.0 # 质量系数
c = 2 * ξ1 * w1 # 阻尼系数
k = w1**2 # 刚度系数
p = F1 / M1 # 外荷载
p = np.insert(p, 0, 0)

v = 0 # 初始时刻位移
v1 = 0 # 初始时刻速度
h = 0.1 # 时间间隔
v_solution = [v]
v1_solution = [v1]
v2_solution = []
for i in range(0,6000):
    v2 = (p[i] - c * v1 - k * v) / m
    p_range = p[i+1]- p[i] 
    kd = k + 6 * m / (h ** 2) + 3 * c / h
    pd = p_range + m * (6 * v1 / h + 3 * v2) + c * (3 * v1 + h * v2 / 2)
    v_range = pd / kd
    v1_range = 3 * v_range / h - 3 * v1 - h * v2 / 2
    v = v + v_range
    v1 = v1 + v1_range
    v_solution.append(v)
    v1_solution.append(v1)
    v2_solution.append(v2)
del v_solution[0]
v_solution = np.array(v_solution).reshape(-1,1)
phi1_z = phi1_z.reshape(-1,1)
v_all = phi1_z @ v_solution.T

print(f"使用时域分析法计算得出地王大厦顶部位移极值为: {max(v_solution)} m")

plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
plt.figure(figsize=(10, 5))
plt.plot(t,v_all[9,:],color = 'red',linestyle = '-')
plt.xlabel('时间（s）')  
plt.ylabel('顶点极值响应（m）')
plt.show()





